In [1]:
# Step 1: Download the dataset
!wget https://azurepublicdatasettraces.blob.core.windows.net/azurepublicdatasetv2/azurefunctions_dataset2019/azurefunctions-dataset2019.tar.xz

# Step 2: Extract the .tar.xz file
!tar -xf azurefunctions-dataset2019.tar.xz

# (Optional) Step 3: Verify extraction
!ls -lh


--2025-09-07 05:41:49--  https://azurepublicdatasettraces.blob.core.windows.net/azurepublicdatasetv2/azurefunctions_dataset2019/azurefunctions-dataset2019.tar.xz
Resolving azurepublicdatasettraces.blob.core.windows.net (azurepublicdatasettraces.blob.core.windows.net)... 52.239.236.228
Connecting to azurepublicdatasettraces.blob.core.windows.net (azurepublicdatasettraces.blob.core.windows.net)|52.239.236.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 142968140 (136M) [application/octet-stream]
Saving to: ‘azurefunctions-dataset2019.tar.xz’

azurefunctions-data 100%[===================>] 136.34M  8.28MB/s    in 12s     

2025-09-07 05:42:01 (11.0 MB/s) - ‘azurefunctions-dataset2019.tar.xz’ saved [142968140/142968140]

total 2.2G
-rwxrwxrwx 1 1000 1000 2.9M Jun 17  2020 app_memory_percentiles.anon.d01.csv
-rwxrwxrwx 1 1000 1000 2.9M Jun 17  2020 app_memory_percentiles.anon.d02.csv
-rwxrwxrwx 1 1000 1000 2.9M Jun 17  2020 app_memory_percentiles.anon.d03.csv
-

# Owner

In [2]:
%pip install duckdb


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 591.3 kB/s  0:01:11m0:00:0100:02
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os, re, duckdb

INPUT_DIR = "./"
OUT_CSV   = "owners.csv"

def sorted_files(prefix):
    fs = [f for f in os.listdir(INPUT_DIR) if f.startswith(prefix) and f.endswith(".csv")]
    return sorted(fs, key=lambda x: int(re.search(r'd(\d+)\.csv$', x).group(1)))

con = duckdb.connect(":memory:")
con.execute("CREATE TEMP TABLE owners_stg(hash_owner TEXT);")

families = [
    "invocations_per_function",            # HashOwner موجوده
    "function_durations_percentiles",      # HashOwner موجوده
    "app_memory_percentiles"               # HashOwner موجوده
]

total_files = 0
for fam in families:
    files = sorted_files(fam)
    total_files += len(files)
    for f in files:
        path = os.path.join(INPUT_DIR, f)
        # فقط HashOwner رو بخون؛ DuckDB فقط همون یک ستون رو اسکن می‌کنه
        con.execute(f"""
            INSERT INTO owners_stg
            SELECT DISTINCT HashOwner
            FROM read_csv_auto('{path}', HEADER=TRUE, SAMPLE_SIZE=-1)
            WHERE HashOwner IS NOT NULL;
        """)

print(f"[owners] scanned files: {total_files}")

# یکتا و خروجی
con.execute("CREATE TEMP TABLE owners_final AS SELECT DISTINCT hash_owner FROM owners_stg;")
con.execute(f"COPY owners_final TO '{OUT_CSV}' (FORMAT CSV, HEADER, DELIMITER ',');")
rows = con.execute("SELECT COUNT(*) FROM owners_final").fetchone()[0]
print(f"[owners] wrote {OUT_CSV} rows={rows:,}")


[owners] scanned files: 40
[owners] wrote owners.csv rows=15,097


# Apps

In [4]:
import os, re, duckdb

INPUT_DIR = "./"
OUT_CSV   = "apps.csv"

def sorted_files(prefix):
    fs = [f for f in os.listdir(INPUT_DIR) if f.startswith(prefix) and f.endswith(".csv")]
    return sorted(fs, key=lambda x: int(re.search(r'd(\d+)\.csv$', x).group(1)))

con = duckdb.connect(":memory:")
con.execute("CREATE TEMP TABLE apps_stg(hash_app TEXT, hash_owner TEXT);")

families = [
    "invocations_per_function",
    "function_durations_percentiles",
    "app_memory_percentiles"
]

total_files = 0
for fam in families:
    files = sorted_files(fam)
    total_files += len(files)
    for f in files:
        path = os.path.join(INPUT_DIR, f)
        con.execute(f"""
            INSERT INTO apps_stg
            SELECT DISTINCT HashApp, HashOwner
            FROM read_csv_auto('{path}', HEADER=TRUE, SAMPLE_SIZE=-1)
            WHERE HashApp IS NOT NULL AND HashOwner IS NOT NULL;
        """)

print(f"[apps] scanned files: {total_files}")

con.execute("""
    CREATE TEMP TABLE apps_final AS
    SELECT DISTINCT hash_app, hash_owner FROM apps_stg;
""")
con.execute(f"COPY apps_final TO '{OUT_CSV}' (FORMAT CSV, HEADER, DELIMITER ',');")
rows = con.execute("SELECT COUNT(*) FROM apps_final").fetchone()[0]
print(f"[apps] wrote {OUT_CSV} rows={rows:,}")

[apps] scanned files: 40
[apps] wrote apps.csv rows=24,964


# Functions → functions_id

In [5]:
import os, re, duckdb

INPUT_DIR = "./"
OUT_CSV = "functions.csv"
FUNCTION_MAPPING = "function_mapping.csv"

def sorted_files(prefix):
    fs = [f for f in os.listdir(INPUT_DIR) if f.startswith(prefix) and f.endswith(".csv")]
    return sorted(fs, key=lambda x: int(re.search(r'd(\d+)\.csv$', x).group(1)))

con = duckdb.connect(":memory:")

# ایجاد جدول نگاشت توابع
print("[mapping] Creating function ID mapping...")
# ✅ نسخه اصلاح شده:
con.execute("""
    CREATE TEMP TABLE function_mapping AS
    WITH all_functions AS (
        SELECT DISTINCT HashFunction as hash_function
        FROM read_csv_auto('./invocations_per_function*.csv', HEADER=TRUE)
        WHERE HashFunction IS NOT NULL
        UNION ALL
        SELECT DISTINCT HashFunction as hash_function
        FROM read_csv_auto('./function_durations_percentiles*.csv', HEADER=TRUE)
        WHERE HashFunction IS NOT NULL
    ),
    unique_functions AS (
        SELECT DISTINCT hash_function 
        FROM all_functions
    )
    SELECT 
        row_number() OVER (ORDER BY hash_function) as function_id,
        hash_function
    FROM unique_functions;
""")

# ذخیره mapping برای استفاده‌های بعدی
con.execute(f"""
    COPY function_mapping TO '{FUNCTION_MAPPING}' (FORMAT CSV, HEADER, DELIMITER ',');
""")
print(f"[mapping] Function mapping saved to {FUNCTION_MAPPING}")

# ایجاد جداول موقت
con.execute("CREATE TEMP TABLE f_inv(function_id BIGINT, hash_app TEXT, trigger TEXT);")
con.execute("CREATE TEMP TABLE f_dur(function_id BIGINT, hash_app TEXT, trigger TEXT);")

# پردازش فایل‌های invocations با استفاده از function_id
inv_files = sorted_files("invocations_per_function")
for f in inv_files:
    path = os.path.join(INPUT_DIR, f)
    con.execute(f"""
        INSERT INTO f_inv
        SELECT DISTINCT 
            m.function_id,
            src.HashApp, 
            src.Trigger
        FROM read_csv_auto('{path}', HEADER=TRUE, SAMPLE_SIZE=-1) AS src
        JOIN function_mapping m ON src.HashFunction = m.hash_function
        WHERE src.HashFunction IS NOT NULL AND src.HashApp IS NOT NULL;
    """)

# پردازش فایل‌های durations با استفاده از function_id
dur_files = sorted_files("function_durations_percentiles")
for f in dur_files:
    path = os.path.join(INPUT_DIR, f)
    con.execute(f"""
        INSERT INTO f_dur
        SELECT DISTINCT 
            m.function_id,
            src.HashApp, 
            NULL AS Trigger
        FROM read_csv_auto('{path}', HEADER=TRUE, SAMPLE_SIZE=-1) AS src
        JOIN function_mapping m ON src.HashFunction = m.hash_function
        WHERE src.HashFunction IS NOT NULL AND src.HashApp IS NOT NULL;
    """)

# ادغام و خروجی نهایی
con.execute("""
    CREATE TEMP TABLE functions_final AS
    SELECT function_id, hash_app, trigger FROM f_inv
    UNION
    SELECT function_id, hash_app, trigger FROM f_dur;
""")

con.execute(f"COPY functions_final TO '{OUT_CSV}' (FORMAT CSV, HEADER, DELIMITER ',');")
rows = con.execute("SELECT COUNT(*) FROM functions_final").fetchone()[0]
print(f"[functions] wrote {OUT_CSV} rows={rows:,}")

[mapping] Creating function ID mapping...
[mapping] Function mapping saved to function_mapping.csv
[functions] wrote functions.csv rows=156,722


# fact_function_duration_daily

In [6]:
import os, re, csv, duckdb

INPUT_DIR = "./"
OUT_FINAL = "fact_function_duration_daily.csv"
TMP_DIR = "./_tmp_dur"
FUNCTION_MAPPING = "function_mapping.csv"

os.makedirs(TMP_DIR, exist_ok=True)
if os.path.exists(OUT_FINAL):
    os.remove(OUT_FINAL)

def sorted_files(prefix):
    fs = [f for f in os.listdir(INPUT_DIR) if f.startswith(prefix) and f.endswith(".csv")]
    return sorted(fs, key=lambda x: int(re.search(r'd(\d+)\.csv$', x).group(1)))

def concat_csv_parts(in_dir, out_path):
    parts = sorted([os.path.join(in_dir, f) for f in os.listdir(in_dir) if f.endswith(".csv")])
    if not parts:
        open(out_path, "w", encoding="utf-8").close()
        return
    with open(out_path, "w", newline="", encoding="utf-8") as fout:
        writer = None
        for i, part in enumerate(parts, 1):
            with open(part, "r", newline="", encoding="utf-8") as fin:
                reader = csv.reader(fin)
                header = next(reader, None)
                if header is None:
                    continue
                if writer is None:
                    writer = csv.writer(fout)
                    writer.writerow(header)
                for row in reader:
                    writer.writerow(row)

con = duckdb.connect(":memory:")

# لود کردن mapping table - اصلاح شده
con.execute(f"""
    CREATE TEMP TABLE function_mapping AS
    SELECT function_id, hash_function
    FROM read_csv_auto('{FUNCTION_MAPPING}', HEADER=TRUE);
""")

dur_files = sorted_files("function_durations_percentiles")
print(f"[durations] {len(dur_files)} files found")

for idx, f in enumerate(dur_files, 1):
    day = int(re.search(r'd(\d+)\.csv$', f).group(1))
    path = os.path.join(INPUT_DIR, f)
    print(f"[{idx}] {f} -> day={day}")

    day_out = os.path.join(TMP_DIR, f"fact_function_duration_daily.d{day:02d}.csv")
    if os.path.exists(day_out):
        os.remove(day_out)

    con.execute(f"""
        COPY (
            SELECT
                m.function_id::BIGINT                 AS function_id,
                {day}::INTEGER                         AS day,
                CAST(Average AS DOUBLE)                AS avg_ms,
                CAST(Minimum AS DOUBLE)                AS min_ms,
                CAST(Maximum AS DOUBLE)                AS max_ms,
                CAST(percentile_Average_0   AS DOUBLE) AS p0,
                CAST(percentile_Average_1   AS DOUBLE) AS p1,
                CAST(percentile_Average_25  AS DOUBLE) AS p25,
                CAST(percentile_Average_50  AS DOUBLE) AS p50,
                CAST(percentile_Average_75  AS DOUBLE) AS p75,
                CAST(percentile_Average_99  AS DOUBLE) AS p99,
                CAST(percentile_Average_100 AS DOUBLE) AS p100,
                CAST(Count AS BIGINT)                 AS count
            FROM read_csv_auto('{path}', HEADER=TRUE, SAMPLE_SIZE=-1) AS src
            JOIN function_mapping m ON src.HashFunction = m.hash_function
        )
        TO '{day_out}' (FORMAT CSV, HEADER, DELIMITER ',');
    """)
    n = con.execute(f"SELECT COUNT(*) FROM read_csv_auto('{path}', HEADER=TRUE, SAMPLE_SIZE=-1)").fetchone()[0]
    print(f"    └─ wrote {n:,} rows → {os.path.basename(day_out)}")

print("[merge] concatenating day parts → final CSV…")
concat_csv_parts(TMP_DIR, OUT_FINAL)
print(f"[done] {OUT_FINAL}")

[durations] 14 files found
[1] function_durations_percentiles.anon.d01.csv -> day=1
    └─ wrote 49,728 rows → fact_function_duration_daily.d01.csv
[2] function_durations_percentiles.anon.d02.csv -> day=2
    └─ wrote 50,112 rows → fact_function_duration_daily.d02.csv
[3] function_durations_percentiles.anon.d03.csv -> day=3
    └─ wrote 50,333 rows → fact_function_duration_daily.d03.csv
[4] function_durations_percentiles.anon.d04.csv -> day=4
    └─ wrote 50,915 rows → fact_function_duration_daily.d04.csv
[5] function_durations_percentiles.anon.d05.csv -> day=5
    └─ wrote 49,916 rows → fact_function_duration_daily.d05.csv
[6] function_durations_percentiles.anon.d06.csv -> day=6
    └─ wrote 39,246 rows → fact_function_duration_daily.d06.csv
[7] function_durations_percentiles.anon.d07.csv -> day=7
    └─ wrote 38,778 rows → fact_function_duration_daily.d07.csv
[8] function_durations_percentiles.anon.d08.csv -> day=8
    └─ wrote 50,327 rows → fact_function_duration_daily.d08.csv
[9] f

# fact_app_memory_daily

In [13]:
import pandas as pd

df = pd.read_csv("./fact_function_duration_daily.csv")
duplicates = df[df.duplicated(subset=['function_id','day','count'], keep=False)]
print(duplicates)

Empty DataFrame
Columns: [function_id, day, avg_ms, min_ms, max_ms, p0, p1, p25, p50, p75, p99, p100, count]
Index: []


In [14]:
import os, re, csv, duckdb

# ---------------------------
# Configuration
# ---------------------------
INPUT_DIR = "./"
OUT_FINAL = "../data/fact_function_duration_daily.csv"
TMP_DIR = "./_tmp_dur"
FUNCTION_MAPPING = "function_mapping.csv"

os.makedirs(TMP_DIR, exist_ok=True)
if os.path.exists(OUT_FINAL):
    os.remove(OUT_FINAL)

# ---------------------------
# Helper functions
# ---------------------------
def sorted_files(prefix):
    """Return files sorted by the number in the filename, e.g., d01.csv, d02.csv"""
    fs = [f for f in os.listdir(INPUT_DIR) if f.startswith(prefix) and f.endswith(".csv")]
    return sorted(fs, key=lambda x: int(re.search(r'd(\d+)\.csv$', x).group(1)))

def concat_csv_parts(in_dir, out_path):
    """Concatenate CSV parts in order into a final CSV"""
    parts = sorted([os.path.join(in_dir, f) for f in os.listdir(in_dir) if f.endswith(".csv")])
    if not parts:
        open(out_path, "w", encoding="utf-8").close()
        return

    with open(out_path, "w", newline="", encoding="utf-8") as fout:
        writer = None
        for part in parts:
            with open(part, "r", newline="", encoding="utf-8") as fin:
                reader = csv.reader(fin)
                header = next(reader, None)
                if header is None:
                    continue
                if writer is None:
                    writer = csv.writer(fout)
                    writer.writerow(header)
                for row in reader:
                    writer.writerow(row)

# ---------------------------
# DuckDB connection
# ---------------------------
con = duckdb.connect(":memory:")

# Load function mapping table
con.execute(f"""
    CREATE TEMP TABLE function_mapping AS
    SELECT function_id, hash_function
    FROM read_csv_auto('{FUNCTION_MAPPING}', HEADER=TRUE);
""")

# ---------------------------
# Process daily function durations
# ---------------------------
dur_files = sorted_files("function_durations_percentiles")
print(f"[durations] {len(dur_files)} files found")

for idx, f in enumerate(dur_files, 1):
    day = int(re.search(r'd(\d+)\.csv$', f).group(1))
    path = os.path.join(INPUT_DIR, f)
    print(f"[{idx}] {f} -> day={day}")

    day_out = os.path.join(TMP_DIR, f"fact_function_duration_daily.d{day:02d}.csv")
    if os.path.exists(day_out):
        os.remove(day_out)

    # Aggregate by function_id and day
    con.execute(f"""
        COPY (
            SELECT
                m.function_id::BIGINT AS function_id,
                {day}::INTEGER AS day,
                AVG(CAST(Average AS DOUBLE)) AS avg_ms,
                MIN(CAST(Minimum AS DOUBLE)) AS min_ms,
                MAX(CAST(Maximum AS DOUBLE)) AS max_ms,
                AVG(CAST(percentile_Average_0   AS DOUBLE)) AS p0,
                AVG(CAST(percentile_Average_1   AS DOUBLE)) AS p1,
                AVG(CAST(percentile_Average_25  AS DOUBLE)) AS p25,
                AVG(CAST(percentile_Average_50  AS DOUBLE)) AS p50,
                AVG(CAST(percentile_Average_75  AS DOUBLE)) AS p75,
                AVG(CAST(percentile_Average_99  AS DOUBLE)) AS p99,
                AVG(CAST(percentile_Average_100 AS DOUBLE)) AS p100,
                SUM(CAST(Count AS BIGINT)) AS count
            FROM read_csv_auto('{path}', HEADER=TRUE, SAMPLE_SIZE=-1) AS src
            JOIN function_mapping m ON src.HashFunction = m.hash_function
            GROUP BY m.function_id,day
        )
        TO '{day_out}' (FORMAT CSV, HEADER, DELIMITER ',');
    """)

    n = con.execute(f"SELECT COUNT(*) FROM read_csv_auto('{path}', HEADER=TRUE, SAMPLE_SIZE=-1)").fetchone()[0]
    print(f"    └─ wrote {n:,} rows → {os.path.basename(day_out)}")

# ---------------------------
# Concatenate all daily CSVs into the final CSV
# ---------------------------
print("[merge] concatenating day parts → final CSV…")
concat_csv_parts(TMP_DIR, OUT_FINAL)
print(f"[done] {OUT_FINAL}")


[durations] 14 files found
[1] function_durations_percentiles.anon.d01.csv -> day=1
    └─ wrote 49,728 rows → fact_function_duration_daily.d01.csv
[2] function_durations_percentiles.anon.d02.csv -> day=2
    └─ wrote 50,112 rows → fact_function_duration_daily.d02.csv
[3] function_durations_percentiles.anon.d03.csv -> day=3
    └─ wrote 50,333 rows → fact_function_duration_daily.d03.csv
[4] function_durations_percentiles.anon.d04.csv -> day=4
    └─ wrote 50,915 rows → fact_function_duration_daily.d04.csv
[5] function_durations_percentiles.anon.d05.csv -> day=5
    └─ wrote 49,916 rows → fact_function_duration_daily.d05.csv
[6] function_durations_percentiles.anon.d06.csv -> day=6
    └─ wrote 39,246 rows → fact_function_duration_daily.d06.csv
[7] function_durations_percentiles.anon.d07.csv -> day=7
    └─ wrote 38,778 rows → fact_function_duration_daily.d07.csv
[8] function_durations_percentiles.anon.d08.csv -> day=8
    └─ wrote 50,327 rows → fact_function_duration_daily.d08.csv
[9] f

# fact_invocations_minutely_sparse

In [8]:
import os
import re
import csv
import duckdb

INPUT_DIR = "./"
OUT_FINAL = "../data/fact_invocations_minutely_sparse.csv"
TMP_DIR = "./_tmp_inv"
FUNCTION_MAPPING = "../data/function_mapping.csv"

os.makedirs(TMP_DIR, exist_ok=True)
if os.path.exists(OUT_FINAL):
    os.remove(OUT_FINAL)

def sorted_files(prefix: str):
    """Return list of files with prefix dXX.csv sorted by day number"""
    fs = [f for f in os.listdir(INPUT_DIR) if f.startswith(prefix) and f.endswith(".csv")]
    return sorted(fs, key=lambda x: int(re.search(r'd(\d+)\.csv$', x).group(1)))

def concat_csv_parts(d: str, o: str):
    """Concatenate CSV parts into one file"""
    parts = sorted([os.path.join(d, f) for f in os.listdir(d) if f.endswith(".csv")])
    with open(o, "w", newline="", encoding="utf-8") as fout:
        writer = None
        for p in parts:
            with open(p, "r", newline="", encoding="utf-8") as fin:
                reader = csv.reader(fin)
                header = next(reader, None)
                if header and writer is None:
                    writer = csv.writer(fout)
                    writer.writerow(header)
                for row in reader:
                    writer.writerow(row)

# Connect to DuckDB
con = duckdb.connect(":memory:")

# Load mapping once
con.execute(f"""
    CREATE TEMP TABLE function_mapping AS
    SELECT function_id, hash_function
    FROM read_csv_auto('{FUNCTION_MAPPING}', HEADER=TRUE);
""")

files = sorted_files("invocations_per_function")
print(f"[invocations] {len(files)} files found")

# Build the minute column list for unpivot
minute_cols = ", ".join(f'"{i}"' for i in range(1, 1441))

for idx, f in enumerate(files, 1):
    day = int(re.search(r'd(\d+)\.csv$', f).group(1))
    path = os.path.join(INPUT_DIR, f)
    print(f"[{idx}] {f} -> day={day}")

    day_out = os.path.join(TMP_DIR, f"fact_invocations_minutely_sparse.d{day:02d}.csv")
    if os.path.exists(day_out):
        os.remove(day_out)

    # Main query: unpivot, join, fill zeros, aggregate into array
    con.execute(f"""
    COPY (
        WITH minutes AS (
            SELECT i AS minute FROM range(1,1441) AS t(i)
        ),
        src AS (
            SELECT * FROM read_csv_auto('{path}', HEADER=TRUE)
        ),
        unpivoted AS (
            SELECT
                u.HashFunction,
                CAST(u.minute AS INTEGER) AS minute,
                CAST(u.calls AS BIGINT) AS calls
            FROM (
                SELECT * FROM src
                UNPIVOT (calls FOR minute IN ({minute_cols}))
            ) u
        ),
        mapped AS (
            SELECT
                m.function_id::BIGINT AS function_id,
                u.minute,
                u.calls
            FROM unpivoted u
            JOIN function_mapping m
            ON u.HashFunction = m.hash_function
        ),
        functions AS (
            SELECT DISTINCT function_id FROM mapped
        ),
        filled AS (
            SELECT f.function_id,
                   m.minute,
                   COALESCE(mp.calls, 0) AS calls
            FROM functions f
            CROSS JOIN minutes m
            LEFT JOIN mapped mp
            ON f.function_id = mp.function_id AND m.minute = mp.minute
        ),
        aggregated AS (
            SELECT function_id,
                   {day}::INTEGER AS day,
                   ARRAY_AGG(calls ORDER BY minute) AS usage
            FROM filled
            GROUP BY function_id
        )
        SELECT function_id, day, usage FROM aggregated
    )
    TO '{day_out}' (FORMAT CSV, HEADER, DELIMITER ',');
    """)

    n = con.execute(f"""
        WITH src AS (
            SELECT * FROM read_csv_auto('{path}', HEADER=TRUE)
        ),
        unpivoted AS (
            SELECT
                u.HashFunction,
                CAST(u.minute AS INTEGER) AS minute,
                CAST(u.calls AS BIGINT) AS calls
            FROM (
                SELECT * FROM src
                UNPIVOT (calls FOR minute IN ({minute_cols}))
            ) u
        ),
        mapped AS (
            SELECT m.function_id::BIGINT AS function_id,
                   u.minute,
                   u.calls
            FROM unpivoted u
            JOIN function_mapping m ON u.HashFunction = m.hash_function
        )
        SELECT COUNT(*) FROM mapped
    """).fetchone()[0]

    print(f"    └─ wrote {n:,} rows (before zero-fill) → {os.path.basename(day_out)}")

print("[merge] concatenating day parts → final CSV…")
concat_csv_parts(TMP_DIR, OUT_FINAL)
print(f"[done] {OUT_FINAL}")

total_rows = con.execute(f"""
    SELECT COUNT(*) FROM read_csv_auto('{OUT_FINAL}', HEADER=TRUE)
""").fetchone()[0]
print(f"[stats] Final CSV has {total_rows:,} rows")


[invocations] 14 files found
[1] invocations_per_function_md.anon.d01.csv -> day=1
    └─ wrote 66,833,280 rows (before zero-fill) → fact_invocations_minutely_sparse.d01.csv
[2] invocations_per_function_md.anon.d02.csv -> day=2
    └─ wrote 67,520,160 rows (before zero-fill) → fact_invocations_minutely_sparse.d02.csv
[3] invocations_per_function_md.anon.d03.csv -> day=3
    └─ wrote 67,796,640 rows (before zero-fill) → fact_invocations_minutely_sparse.d03.csv
[4] invocations_per_function_md.anon.d04.csv -> day=4
    └─ wrote 68,438,880 rows (before zero-fill) → fact_invocations_minutely_sparse.d04.csv
[5] invocations_per_function_md.anon.d05.csv -> day=5
    └─ wrote 67,186,080 rows (before zero-fill) → fact_invocations_minutely_sparse.d05.csv
[6] invocations_per_function_md.anon.d06.csv -> day=6
    └─ wrote 52,457,760 rows (before zero-fill) → fact_invocations_minutely_sparse.d06.csv
[7] invocations_per_function_md.anon.d07.csv -> day=7
    └─ wrote 51,884,640 rows (before zero-fill)

# File sizes

In [10]:
import os

def get_file_size(file_path):
    """Get file size in appropriate units"""
    if not os.path.exists(file_path):
        return "0.00 MB (File not found)"

    size_bytes = os.path.getsize(file_path)

    # Convert to different units
    if size_bytes >= 1024 * 1024 * 1024:
        size_gb = size_bytes / (1024 * 1024 * 1024)
        return f"{size_gb:.2f} GB"
    elif size_bytes >= 1024 * 1024:
        size_mb = size_bytes / (1024 * 1024)
        return f"{size_mb:.2f} MB"
    elif size_bytes >= 1024:
        size_kb = size_bytes / 1024
        return f"{size_kb:.2f} KB"
    else:
        return f"{size_bytes} B"

def print_all_file_sizes():
    pre = "../data/"
    """Print size of all output files"""
    files = [
        "owners.csv",
        "apps.csv",
        "functions.csv",
        "function_mapping.csv",
        "fact_function_duration_daily.csv",
        "fact_app_memory_daily.csv",
        "fact_invocations_minutely_sparse.csv"
    ]

    print("📊 File Sizes:")
    print("=" * 60)

    total_size_bytes = 0

    for file in files:
        file = pre + file
        if os.path.exists(file):
            size_bytes = os.path.getsize(file)
            total_size_bytes += size_bytes
            size_str = get_file_size(file)
            print(f"📁 {file:35} → {size_str}")
        else:
            print(f"❌ {file:35} → File not found")

    print("=" * 60)

    # Calculate total size
    total_mb = total_size_bytes / (1024 * 1024)
    total_gb = total_size_bytes / (1024 * 1024 * 1024)
    print(f"💾 Total size: {total_mb:.2f} MB ({total_gb:.2f} GB)")

# Execute the function
print_all_file_sizes()

📊 File Sizes:
📁 ../data/owners.csv                  → 958.32 KB
📁 ../data/apps.csv                    → 3.09 MB
📁 ../data/functions.csv               → 11.12 MB
📁 ../data/function_mapping.csv        → 5.62 MB
📁 ../data/fact_function_duration_daily.csv → 47.29 MB
📁 ../data/fact_app_memory_daily.csv   → 25.05 MB
📁 ../data/fact_invocations_minutely_sparse.csv → 2.53 GB
💾 Total size: 2684.28 MB (2.62 GB)


In [9]:


import csv
pre = "../data/"
l = {"owners.csv" , "apps.csv" , "functions.csv" , "function_mapping.csv" , "fact_app_memory_daily.csv" , "fact_function_duration_daily.csv"
     ,"fact_invocations_minutely_sparse.csv"}
for x in l:
    with open(pre+x) as csv_file:
        csv_reader = csv.DictReader(csv_file)
        header = list(next(csv_reader).keys())

    print(f"{x}:", header)



owners.csv: ['hash_owner']
fact_invocations_minutely_sparse.csv: ['function_id', 'day', 'usage']
fact_app_memory_daily.csv: ['hash_app', 'day', 'sample_count', 'avg_mb', 'p1', 'p5', 'p25', 'p50', 'p75', 'p95', 'p99', 'p100']
functions.csv: ['function_id', 'hash_app', 'trigger']
apps.csv: ['hash_app', 'hash_owner']
fact_function_duration_daily.csv: ['function_id', 'day', 'avg_ms', 'min_ms', 'max_ms', 'p0', 'p1', 'p25', 'p50', 'p75', 'p99', 'p100', 'count']
function_mapping.csv: ['function_id', 'hash_function']


In [6]:
%pip install pandas

  Using cached pandas-2.3.2-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (91 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.3.2-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.1 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 952.1 kB/s  0:00:18 eta 0:00:01
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [pandas]2m3/4 [pandas]
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd

df = pd.read_csv("../data/functions.csv")
df['trigger'] = df['trigger'].fillna('unknown')  # or any default string
df.to_csv("functions_clean.csv", index=False)